In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from FrCHFM import compute_frchfm
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

[NbConvertApp] Converting notebook FrCHFM.ipynb to python
[NbConvertApp] Writing 9094 bytes to FrCHFM.py


In [2]:
def visualize_frchfm_matrices(original_frchfm, rotated_frchfm, angle):
    """
    Visualizes the original and rotated FrCHFM matrices for comparison.
    
    Parameters:
        original_frchfm (2D numpy array): FrCHFM matrix of the original image.
        rotated_frchfm (2D numpy array): FrCHFM matrix of the rotated image.
        angle (float): Rotation angle for labeling purposes.
    """
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle(f"FrCHFM Comparison for Rotation {angle}°")
    
    axs[0].imshow(np.abs(original_frchfm), cmap='viridis')
    axs[0].set_title("Original FrCHFM Magnitude")
    axs[0].axis('off')
    
    axs[1].imshow(np.abs(rotated_frchfm), cmap='viridis')
    axs[1].set_title(f"Rotated FrCHFM Magnitude ({angle}°)")
    axs[1].axis('off')
    
    plt.show()

In [3]:
def test_frchfm_rotation_invariance_with_visuals(image, angles=[45, 90, 180]):
    """
    Tests and visualizes the rotation invariance of Fractional Chebyshev-Fourier Moments (FrCHFM).
    
    Parameters:
        image (numpy array): The original image to test.
        calculate_frchfm (function): Function to calculate FrCHFM of an image.
        angles (list): List of angles to rotate the image for testing.
        
    Returns:
        results (dict): Dictionary where keys are angles and values are bools indicating invariance.
    """

    frchfm_original = calculate_frchfm(image)
    
    results = {}
    
    # Plot the original image
    plt.imshow(image, cmap='gray')
    plt.title("Original Image")
    plt.axis('off')
    plt.show()

    for angle in angles:
        # Rotate the image
        rotated_image = rotate_image_fixed_size(image, angle)
        
        # Plot the rotated image
        plt.imshow(rotated_image, cmap='gray')
        plt.title(f"Rotated Image ({angle}°)")
        plt.axis('off')
        plt.show()
        
        # Calculate FrCHFM for the rotated image
        frchfm_rotated = calculate_frchfm(rotated_image)
        
        # Visualize FrCHFM matrices for comparison
        visualize_frchfm_matrices(frchfm_original, frchfm_rotated, angle)
        
        # Compare the original and rotated FrCHFM matrices
        invariant = np.allclose(np.abs(frchfm_original), np.abs(frchfm_rotated), atol=1e-6)
        
        # Store the result for this angle
        results[angle] = invariant
    
    return results

In [4]:
def rotate_image_fixed_size(image, angle):

    """
    Rotates the image by a given angle around its center, maintaining the original size with potential cropping.
    Parameters:

        image (numpy array): Input image to rotate.

        angle (float): Angle by which to rotate the image.
    Returns:
        rotated_image (numpy array): Rotated image, constrained to original dimensions.
    """
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Compute the rotation matrix and apply rotation
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, M, (w, h))

    return rotated_image


In [5]:
def compute_frchfm_difference(frchfm_original, frchfm_rotated):
    """
    Calculates the mean absolute difference between the magnitudes of the original and rotated FrCHFM matrices.
    
    Parameters:
        frchfm_original (2D numpy array): FrCHFM matrix of the original image.
        frchfm_rotated (2D numpy array): FrCHFM matrix of the rotated image.
        
    Returns:
        mean_difference (float): Mean absolute difference between the magnitudes.
    """
    return np.mean(np.abs(np.abs(frchfm_original) - np.abs(frchfm_rotated)))

In [6]:
def rotate_and_compare(image, calculate_frchfm, angles=[30, 45, 90, 180]):
    """
    Rotates an image by specified angles, computes FrCHFM for each, and measures rotation invariance.
    
    Parameters:
        image (numpy array): The original image to test.
        calculate_frchfm (function): Function to calculate FrCHFM of an image.
        angles (list): List of angles to rotate the image for testing.
        
    Returns:
        results (dict): A dictionary where keys are angles and values are tuples (is_invariant, deviation).
                        - `is_invariant` (bool): Whether the FrCHFM is invariant at this angle.
                        - `deviation` (float): Mean absolute difference in FrCHFM magnitudes.
    """
    max_degree = 5  # Change depending on desired precision
    fractional_parameter_t = 0.5  # Fractional parameter t
    frchfm_original = calculate_frchfm(image)
    results = {}

    for angle in angles:
        rotated_image = rotate_image_fixed_size(image, angle)
        frchfm_rotated = calculate_frchfm(rotated_image)
        
        # Calculate the mean absolute difference as a measure of deviation
        deviation = compute_frchfm_difference(frchfm_original, frchfm_rotated)
        
        # Determine invariance by checking if deviation is within tolerance
        is_invariant = deviation < 1e-6
        results[angle] = (is_invariant, deviation)
    
    return results

In [7]:
def test_rotation_invariance_parallel(images, calculate_frchfm, angles=[30, 45, 90, 180]):
    """
    Tests rotation invariance for multiple images in parallel and calculates deviations, with a progress indicator.
    
    Parameters:
        images (list of numpy arrays): List of images to test.
        calculate_frchfm (function): Function to calculate FrCHFM of an image.
        angles (list): List of angles to rotate each image for testing.
        
    Returns:
        overall_results (dict): A dictionary where keys are image indices and values are dictionaries
                                of angle results, each with (is_invariant, deviation).
    """
    overall_results = {}
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(rotate_and_compare, img, calculate_frchfm, angles): idx for idx, img in enumerate(images)}
        
        # Use tqdm to show progress
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Images"):
            idx = futures[future]
            try:
                result = future.result()
                overall_results[idx] = result
            except Exception as e:
                print(f"Image {idx} failed with exception: {e}")
    
    return overall_results


In [ ]:
# Directory containing the images
image_folder = "cs4485-images"

# Define parameters for calculate_frchfm
max_degree = 5
fractional_parameter_t = 0.5

def calculate_frchfm(x):
    return compute_frchfm(x, max_degree, fractional_parameter_t)

# Load all images from the folder
images = [cv2.imread(os.path.join(image_folder, filename), cv2.IMREAD_GRAYSCALE)
          for filename in os.listdir(image_folder) if filename.endswith(('.png', '.jpg', '.jpeg'))]

# Run the parallel rotation invariance test
results = test_rotation_invariance_parallel(images, calculate_frchfm)

# Print the results
print(results)

Processing Images:   6%|███▉                                                          | 16/256 [01:12<18:04,  4.52s/it]
